In [1]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

--2023-12-10 16:06:24--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.224.181.60, 13.224.181.95, 13.224.181.117, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.224.181.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  2.00MB/s    in 1.4s    

2023-12-10 16:06:26 (2.00 MB/s) - ‘data.zip’ saved [2882130/2882130]

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/I

In [2]:
!wget https://www.statmt.org/europarl/v7/fr-en.tgz
!tar -xf fr-en.tgz

--2023-12-10 16:06:28--  https://www.statmt.org/europarl/v7/fr-en.tgz
Resolving www.statmt.org (www.statmt.org)... 129.215.32.28
Connecting to www.statmt.org (www.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202718517 (193M) [application/x-gzip]
Saving to: ‘fr-en.tgz’

fr-en.tgz           100%[===================>] 193.33M  12.2MB/s    in 18s     

2023-12-10 16:06:48 (10.7 MB/s) - ‘fr-en.tgz’ saved [202718517/202718517]



In [3]:
!pip install torch

In [4]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import string
import random
import unicodedata
import pickle

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import re
import string
import unicodedata
import pickle
import pandas as pd 


file = open('europarl-v7.fr-en.en', mode='rt', encoding='utf-8')
t = file.read()
file.close()
sentences = t.strip().split('\n')
new_s = []
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    table = str.maketrans('', '', string.punctuation)
    for s in sentences:
        s = unicodedata.normalize('NFD', line).encode('ascii', 'ignore')
        s = s.decode('UTF-8')
        s = s.split()
        s = [w.lower() for w in s]
        s = [w.translate(table) for w in s]
        s = [re_print.sub('', w) for w in s]
        s = [wa for w in s if w.isalpha()]
        new_s.append(' '.join(s))
pickle.dump(new_s, open(filename, 'wb'))
print('Saved: %s' % filename)
for i in range(10):
    print(new_s[i])

file = open('europarl-v7.fr-en.fr', mode='rt', encoding='utf-8')
t = file.read()
file.close()
sentences = t.strip().split('\n')
new_s = []
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    table = str.maketrans('', '', string.punctuation)
    for s in sentences:
        s = unicodedata.normalize('NFD', line).encode('ascii', 'ignore')
        s = s.decode('UTF-8')
        s = s.split()
        s = [w.lower() for w in s]
        s = [w.translate(table) for w in s]
        s = [re_print.sub('', w) for w in s]
        s = [wa for w in s if w.isalpha()]
        new_s.append(' '.join(s))
pickle.dump(new_s, open(filename, 'wb'))
print('Saved: %s' % filename)
for i in range(1):
    print(new_s[i])

with open('french.pkl', 'rb') as f:
    fr = pickle.load(f)

with open('english.pkl', 'rb') as f:
    eng = pickle.load(f)
    
data = pd.DataFrame(zip(eng, fr), columns = ['English', 'French'])
data

Saved: english.pkl
resumption of the session
i declare resumed the session of the european parliament adjourned on friday december and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period
although as you will have seen the dreaded millennium bug failed to materialise still the people in a number of countries suffered a series of natural disasters that truly were dreadful
you have requested a debate on this subject in the course of the next few days during this partsession
in the meantime i should like to observe a minute s silence as a number of members have requested on behalf of all the victims concerned particularly those of the terrible storms in the various countries of the european union
please rise then for this minute s silence
the house rose and observed a minute s silence
madam president on a point of order
you will be aware from the press and television that there have been a number of bomb explosions and killings in sri

,English,French
0,resumption of the session,reprise de la session
1,i declare resumed the session of the european ...,je declare reprise la session du parlement eur...
2,although as you will have seen the dreaded mil...,comme vous avez pu le constater le grand bogue...
3,you have requested a debate on this subject in...,vous avez souhaite un debat a ce sujet dans le...
4,in the meantime i should like to observe a min...,en attendant je souhaiterais comme un certain ...
...,...,...
2007718,i would also like although they are absent to ...,je me permettrai meme bien quils soient absent...
2007719,i am not going to reopen the millennium or not...,je ne rouvrirai pas le debat sur le millenaire...
2007720,adjournment of the session,interruption de la session
2007721,i declare the session of the european parliame...,je declare interrompue la session du parlement...


In [7]:
data2 = pd.read_csv('./data/eng-fra.txt', delimiter='\t', names = ['English', 'French'])
data2

,English,French
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !
...,...,...
135837,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
135838,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
135839,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
135840,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


In [8]:
data = pd.concat([data,data2], ignore_index= True, axis = 0)

data.to_csv('eng-fra.txt')
data

,English,French
0,resumption of the session,reprise de la session
1,i declare resumed the session of the european ...,je declare reprise la session du parlement eur...
2,although as you will have seen the dreaded mil...,comme vous avez pu le constater le grand bogue...
3,you have requested a debate on this subject in...,vous avez souhaite un debat a ce sujet dans le...
4,in the meantime i should like to observe a min...,en attendant je souhaiterais comme un certain ...
...,...,...
2143560,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
2143561,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
2143562,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
2143563,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


In [9]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchtext
from torchtext.data import get_tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
SOS_token = 0
EOS_token = 1

class Voc:
    def __init__(self, name):
        self.name = name
        self.w2i = {}
        self.w2c = {}
        self.i2w = {0: "SOS", 1: "EOS"}
        self.num_of_W = 2 

    def insertSeq(self, seq):
        for word in seq.split(' '):
            self.insertWord(word)

    def insertWord(self, word):
        if word not in self.w2i:
            self.w2i[word] = self.num_of_W
            self.w2c[word] = 1
            self.i2w[self.num_of_W] = word
            self.num_of_W += 1
        else:
            self.w2c[word] += 1

In [11]:
def removeNonLettersAndLowercase(s):
    s = ''.join(c for c in unicodedata.normalize('NFD', s.lower().strip()) if unicodedata.category(c) != 'Mn')
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [12]:
def rVocs(Voc1, Voc2, reverse=False):
    print("Reading lines...")

    lines = open('data/%s-%s.txt' % (Voc1, Voc2), encoding='utf-8').read().strip().split('\n')
    pairs = [[removeNonLettersAndLowercase(s) for s in l.split('\t')] for l in lines]
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        i_Voc = Voc(Voc2)
        o_Voc = Voc(Voc1)
    else:
        i_Voc = Voc(Voc1)
        o_Voc = Voc(Voc2)

    return i_Voc, o_Voc, pairs

In [14]:
def processData(Voc1, Voc2, reverse=False):
    i_Voc, o_Voc, pairs = rVocs(Voc1, Voc2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = [p for p in pairs if len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH]
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        i_Voc.insertSeq(pair[0])
        o_Voc.insertSeq(pair[1])
    print(i_Voc.name, i_Voc.num_of_W)
    print(o_Voc.name, o_Voc.num_of_W)
    return i_Voc, o_Voc, pairs

i_Voc, o_Voc, pairs = processData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 122295 sentence pairs
Counting words...
Counted words:
fra 19507
eng 11759
['presque un millier de personnes ont pris part a la manifestation', 'nearly a thousand people participated in the demonstration']


In [ ]:
MAX_LENGTH = 20

In [15]:
class Encoder(nn.Module):
    def __init__(self, i_s, h_s, dropout_p=0.1):
        super(Encoder, self).__init__()
        self.h_s = h_s
        self.embedding = nn.Embedding(i_s, h_s)
        self.gru = nn.GRU(h_s, h_s, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        o, hidden = self.gru(embedded)
        return o, hidden

In [16]:
class Decoder(nn.Module):
    def __init__(self, h_s, o_s):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(o_s, h_s)
        self.gru = nn.GRU(h_s, h_s, batch_first=True)
        self.out = nn.Linear(h_s, o_s)

    def forward(self, e_o, e_h, t_t=None):
        batch_size = e_o.size(0)
        d_i = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        d_h = e_h
        d_os = []

        for i in range(MAX_LENGTH):
            d_o, d_h  = self.forward(d_i, d_h)
            d_os.append(d_o)

            if t_t is not None:
                d_i = t_t[:, i].unsqueeze(1)
            else:
                _, topi = d_o.topk(1)
                d_i = topi.squeeze(-1).detach()  

        d_os = torch.cat(d_os, dim=1)
        d_os = F.log_softmax(d_os, dim=-1)
        return d_os, d_h, None

    def forward(self, input, hidden):
        o = self.embedding(input)
        o = F.relu(o)
        o, hidden = self.gru(o, hidden)
        o = self.out(o)
        return o, hidden

In [17]:
from sklearn.model_selection import train_test_split

def i_f_s(Voc, sentence):
    return [Voc.w2i[word] for word in sentence.split(' ')]

def t_f_s(Voc, sentence):
    indexes = i_f_s(Voc, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def t_f_p(pair):
    i_t = t_f_s(i_Voc, pair[0])
    t_t = t_f_s(o_Voc, pair[1])
    return (i_t, t_t)

def get_data(batch_size):
    i_Voc, o_Voc, pairs = processData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = i_f_s(i_Voc, inp)
        tgt_ids = i_f_s(o_Voc, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))
    
    train_pairs, test_pairs = train_test_split(train_data, test_size=0.3, random_state=42)
    val_pairs, test_pairs = train_test_split(test_pairs, test_size=0.5, random_state=42)
    print('len of training samples ', len(train_pairs))

    
    train_dataloader = DataLoader(train_pairs, batch_size=batch_size, shuffle= True)
    test_dataloader = DataLoader(test_pairs, batch_size=batch_size, shuffle= True)
    val_dataloader = DataLoader(val_pairs, batch_size=batch_size, shuffle= True)                                        
    return i_Voc, o_Voc, train_dataloader, test_dataloader, val_dataloader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [19]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        i_t, t_t = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        e_o, e_h = encoder(i_t)
        d_os, _, _ = decoder(e_o, e_h, t_t)
        
        _, predicted = d_os.max(dim=2)
        correct = (predicted == t_t)
        accuracy = correct.sum().item() / (t_t.size(0) * t_t.size(1))

        loss = criterion(
            d_os.view(-1, d_os.size(-1)),
            t_t.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader), accuracy

In [20]:
def evaluate_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    #encoder.eval()
    #decoder.eval()
    for data in dataloader:
        i_t, t_t = data

        
        e_o, e_h = encoder(i_t)
        d_os, _, _ = decoder(e_o, e_h, t_t)

        loss = criterion(
            d_os.view(-1, d_os.size(-1)),
            t_t.view(-1)
        )


        total_loss += loss.item()

    return total_loss / len(dataloader)

In [21]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [22]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [23]:
def train(train_dataloader, val_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    plt_val_losses = []
    accuracies = []
    print_loss_total = 0 
    plot_loss_total = 0 
    plt_val_loss = 0
    print_val_loss_total = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss, accuracy = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        val_loss = evaluate_epoch(val_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        print_val_loss_total += val_loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            val_loss_avg = print_val_loss_total / print_every
            print_loss_total = 0
            print_val_loss_total = 0
            print('%s (%d %d %d%% ) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg, val_loss_avg))
            print('training_loss ', print_loss_avg, 'val_loss_avg ', val_loss_avg, ' accuracy ', accuracy)

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plt_val_avg = print_val_loss_total / plot_every
            plt_val_losses.append(plt_val_avg)
            accuracies.append(accuracy)
            plot_loss_total = 0
            print_val_loss_total = 0

    showPlot(plot_losses)
    showPlot(plt_val_losses)
    showPlot(accuracies)

In [24]:
def evaluate(encoder, decoder, input_tensor, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, input_tensor)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [25]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [26]:
batch_size = 32



input_lang, output_lang, train_dataloader, test_pairs, val_dataloader = get_dataloader(batch_size)
test_dataloader = DataLoader(test_pairs, batch_size=batch_size, shuffle= True)

Reading lines...
Read 135842 sentence pairs
Trimmed to 122295 sentence pairs
Counting words...
Counted words:
fra 19507
eng 11759
len of training samples  85606


In [27]:
hidden_size = 128
batch_size = 32



input_lang, output_lang, train_dataloader, test_dataloader, val_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, val_dataloader, encoder, decoder, 55, print_every=1, plot_every=5)

Reading lines...
Read 135842 sentence pairs
Trimmed to 122295 sentence pairs
Counting words...
Counted words:
fra 19507
eng 11759
len of training samples  85606
0m 34s (- 31m 29s) (1 1 2% ) 1.9629
training_loss  2.4682893043409013 val_loss_avg  1.9629274361640319  accuracy  0.7361111111111112
1m 9s (- 30m 42s) (2 3 1% ) 1.6644
training_loss  1.7613649538903315 val_loss_avg  1.6644275373282749  accuracy  0.7777777777777778
1m 43s (- 30m 1s) (3 5 1% ) 1.5096
training_loss  1.4906892455078562 val_loss_avg  1.5096381391797746  accuracy  0.7638888888888888
2m 18s (- 29m 21s) (4 7 1% ) 1.3998
training_loss  1.3060284457399172 val_loss_avg  1.3998025767894573  accuracy  0.7638888888888888
2m 52s (- 28m 41s) (5 9 1% ) 1.3287
training_loss  1.1647089270510125 val_loss_avg  1.328714323895318  accuracy  0.6527777777777778
3m 26s (- 28m 5s) (6 10 1% ) 1.2715
training_loss  1.0510568253261625 val_loss_avg  1.2715442018641827  accuracy  0.875
4m 0s (- 27m 30s) (7 12 0% ) 1.2334
training_loss  0.9575

In [28]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> j etais l homme le plus heureux de la terre
= i was the happiest man on earth
< i was the happiest man on earth <EOS>

> je vais te montrer la ville
= i will show you around the city
< i ll show you the town <EOS>

> ils ont redige un projet de loi sur la sante
= they ve written a bill for health care
< they ve written a foreign trade every day <EOS>

> je suis tres strict
= i m very strict
< i m very strict <EOS>

> cette viande s est gatee
= this meat has gone bad
< this thing is terrible at all different <EOS>

> je vais le mettre dans ma chambre
= i m going to put it in my room
< i ll put my room back <EOS>

> on a donne une medaille a tom
= tom was given a medal
< tom was given a medal <EOS>

> il n est pas poli de montrer les autres du doigt
= it is not polite to point at others
< it is no easy to keep from his attention <EOS>

> je viens d angleterre
= i come from england
< i just changed <EOS>

> je pense que nous ferions mieux d aider tom
= i think we d better help tom
< i t

In [29]:
torch.save(encoder.state_dict(), 'encoder_weights.pth')


In [30]:
torch.save(decoder.state_dict(), 'decoder_weights.pth')